In [1]:
import os
import git2net

print(os.path)

sqlite_db_file = '/home/luc/pip/git2net/ds_gw_1/git2net_tutorial (Kopie).db'

#check, if database file is there; remove if yes
#if os.path.exists(sqlite_db_file):
#    os.remove(sqlite_db_file)
#    print('sqlite_db_file removed!')


#set variable to whatever folder the to-be-mined git repository is located:
repo_dir = '/home/luc/pip/git2net/ds_gw_1/TLP'

<module 'posixpath' from '/home/luc/pip/git2net/pip_ds/lib/python3.7/posixpath.py'>


In [2]:


#mining process: 
#repo_dir: to-be-mined git repository
#sqlite_db_file: to-be-stored values in database

#git2net.mine_git_repo(repo_dir, sqlite_db_file)



In [3]:
git2net.mining_state_summary(repo_dir, sqlite_db_file)

0it [00:00, ?it/s]1097 / 1097 (100.00%) of commits were successfully mined.



,hash,is_merge,modifications,author_name,author_email,author_date


In [4]:
#draws nodes for visualization:
#t = Pathpy temporal network
#node_info = Information about node
#edge_info = Information about edges

t, node_info, edge_info = git2net.get_coediting_network(sqlite_db_file)
t

In [5]:


print(t)
print('\n')
print(node_info)
print('\n')
print(edge_info)
print('\n')



Nodes:			27
Time-stamped links:	585
Links/Nodes:		21.666666666666668
Observation period:	[1331666537, 1570102694]
Observation length:	 238436157 
Time stamps:		 151 
Avg. inter-event dt:	 1589574.38
Min/Max inter-event dt:	 64/24635053


{}


{}




In [6]:
import pathpy as pp

#returns a Graph, with an edge pointing from node A to node B whenever A changed a line of original author B: 
pp.Network.from_temporal_network(t)

In [7]:


#visualize which files the authors collaborated on:
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file)
n = pp.Network.from_temporal_network(t)
n



In [8]:
#colour the author nodes light blue and the collaboration files dark blue:
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_colour = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_colour)

In [9]:


#draw graph showing authors working collaborating on same files:
#all information of direction of edges is lost:
n, node_info, edge_info = git2net.get_coauthorship_network(sqlite_db_file)
n



In [10]:
from datetime import datetime

time_from = datetime(2019, 5, 1)
t, node_info, edge_info = git2net.get_bipartite_network(sqlite_db_file, time_from=time_from)

print(node_info)

n = pp.Network.from_temporal_network(t)
colour_map = {'author': '#73D2DE', 'file': '#2E5EAA'}
node_colour = {node: colour_map[node_info['class'][node]] for node in n.nodes}
pp.visualisation.plot(n, node_color=node_colour)


{'class': {'Thomas Koch': 'author', 'tlp.8': 'file', 'changelog': 'file', 'tlp-func-base.in': 'file', 'tlp.service.in': 'file', 'tlp-stat.in': 'file', '35-tlp-func-batt': 'file', 'default': 'file', 'Makefile': 'file', 'tlp.init': 'file', '10-tlp-func-cpu': 'file', 'tlp-sleep.service.8': 'file', 'tlp.service.8': 'file', 'tlp.bash_completion': 'file', 'tlp-sleep': 'file', 'tlp-sleep.service.in': 'file', 'tlp-func-stat': 'file', 'tlp.in': 'file', 'README.md': 'file', 'linrunner': 'author', '30-tlp-func-rf-sw': 'file', '40-tlp-func-bay': 'file', '15-tlp-func-disk': 'file'}}


In [11]:
import pandas as pd
import sqlite3

#connect to database file:
con = sqlite3.connect(sqlite_db_file)
c = con.cursor()


#create a new table:
query0="""
    CREATE TABLE IF NOT EXISTS df2(
    hash TEXT PRIMARY KEY,
    committer_name TEXT,
    committer_date TEXT,
    original_commit_deletion TEXT,
    levenshtein_dist INTEGER);
"""

#select only rows of table "edits", where edit_type='replacement', 
#merge this with table "commits" along the key 'hash',
#group this by unique hashes, taking the sum of the levenshtein dist of same hashes
#finally, insert this into a new table "df2" to avoid key words as column names:
query1="""
    INSERT INTO df2(hash, committer_name, committer_date, original_commit_deletion, levenshtein_dist)
    SELECT hash, committer_name, committer_date, original_commit_deletion, "SUM(levenshtein_dist)"
    FROM
    (
    SELECT 
    hash,
    committer_name,
    committer_date,
    original_commit_deletion,
    SUM(levenshtein_dist)
    FROM(SELECT * FROM edits INNER JOIN commits ON commits.hash = edits.commit_hash WHERE edit_type = 'replacement')
    GROUP BY hash
    );
"""

#execute the two queries on the database:
c.execute(query0)
c.execute(query1)

#save the queries:
con.commit()

#close the database to make it accessible for others:
con.close()




hash committer_name  \
0  0063dea8e26fca5014cff4087fc778459bb9be75    Thomas Koch   
1  007ff3c1058cef8a059d5a67a791145ef39eb859    Thomas Koch   
2  00d8cb2499bb85346fa62c9eec8631007cb01cca    Thomas Koch   

        committer_date                  original_commit_deletion  \
0  2017-03-24 21:04:37  aa2d2d9b3d3a85a56c6e5ec63fc192d87b258106   
1  2019-04-10 06:25:33  8e59ad6cb17ef50f141e522ad2c1336db6e77500   
2  2016-03-12 17:16:36  caca7f1673c5ec16386835712abcbbc6d447f972   

   levenshtein_dist  
0                81  
1                41  
2                16  
(942, 5)


In [12]:
query01="""
    CREATE TABLE IF NOT EXISTS df3(
    hash TEXT PRIMARY KEY,
    committer_name TEXT,
    committer_date TEXT,
    original_commit_deletion TEXT,
    levenshtein_dist INTEGER);
"""

#sort committer_date ascending:

query2="""
    INSERT INTO df3(hash, committer_name, committer_date, original_commit_deletion, levenshtein_dist)
    SELECT *
    FROM
    (
    SELECT * 
    FROM df2 
    ORDER BY committer_date ASC
    );
"""

query3="""
    DROP TABLE df2;
"""

query4="""
    DROP TABLE df3;
"""


#ordering:
#query0, query2_edited, query3
#then:
#query01, query2_edited, query4


con = sqlite3.connect(sqlite_db_file)
c = con.cursor()

c.execute(query01)
c.execute(query2)
c.execute(query3)
#result: df3

print(c.fetchmany(5))

con.commit()
con.close()



[]


In [13]:
con = sqlite3.connect(sqlite_db_file)

#output df2 as a pandas dataframe
df = pd.read_sql_query("SELECT * FROM df3;", con=con)

print(df.head(3))
print(df.shape)
print(type(df))

hash committer_name  \
0  4b6204169763d76567c846528be692afd30c38e4    Thomas Koch   
1  5d2d58c47f233bce9a6f9a9371ca75301073c6ae    Thomas Koch   
2  f955610e360a98f1d7b1b1f209f5f2550ef8238c    Thomas Koch   

        committer_date                  original_commit_deletion  \
0  2010-01-22 18:16:44  624dc813bd780af99e0d73e31fd2de9f3cc6c9ad   
1  2010-01-29 17:47:37  624dc813bd780af99e0d73e31fd2de9f3cc6c9ad   
2  2010-01-29 17:51:55  5d2d58c47f233bce9a6f9a9371ca75301073c6ae   

   levenshtein_dist  
0                 4  
1                60  
2                 2  
(942, 5)
<class 'pandas.core.frame.DataFrame'>


In [14]:
def adding_up(df):
    print(df)
    print(df.shape[0])
    if df.shape[0] == 1:
        return df
    for i in range(df.shape[0]):
        print('i=',i)        
        for j in range(i+1, df.shape[0]):
            if df.iloc[i]['A'] < df.iloc[j]['A']:
                print(df.iloc[i]['D'])
                print(df.iloc[j]['D'])
                tmp = df.iloc[i]['D'] + df.iloc[j]['D']
                df.iloc[i]['D'] = tmp
                print(df.iloc[i]['D'])
                print('Es wird gedropped, anschließend Indexreset:\n')
                df = df.drop(df.index[j])        
                df = df.reset_index(drop=True)
                print(df)
                #print(type(df))
                return df

In [15]:

print(type(df))

#turn Pandas dataframe into numpy array:
df = df.values

print(type(df))

print(df.shape)

#for i in range(1,df.shape[0]):
    #print(i)
    #print(type(float(df[i][2])))


#print(df)






<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>
(942, 5)


In [16]:
import numpy as np

def add_column(df):
    #add a full-length '0'-column to the dataframe:
    new_column = np.array(np.zeros(df.shape[0]))
    return np.column_stack((df, new_column))

In [17]:
df = add_column(df)

In [18]:
#make date stamps readible, by turning them into a datetime-object:

from datetime import datetime

def date_to_int(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

In [19]:
print(date_to_int(df[0][2]))

2010-01-22 18:16:44


In [20]:
#calculate amount of time (in seconds) since last commit for every authors commit; save values in separate column

for i in range(df.shape[0]-1):
    author = df[i][1]
    commit_time = df[i][2]
    for j in range(i+1,df.shape[0]):
        author_next = df[j][1]
        commit_time_next = df[j][2]
        if author == author_next:
                df[j][5] = (date_to_int(commit_time_next) - date_to_int(commit_time)).total_seconds()
                break
            


In [21]:
df = add_column(df)
print(df.shape)

(942, 7)


In [25]:
print(df[1][5])

603053.0


In [26]:
#calculate a measure for productivity:
#levenshtein_dist / time_between_commits = levenshtein_dist per second

for i in range(df.shape[0]):
    if df[i][5] == 0:
        df[i][6] = 0
    else:     
        df[i][6] = df[i][4] / df[i][5]


In [0]:
df[1][5].seconds